# Introduction
> This is the workbook that we will submit.
> All relevant visuals and data analysis for our report should be produced by executing this workbook's cells in order. 

In [ ]:
'''Install non-standard libraries'''
!pip install skopt

In [ ]:
'''Generic Imports'''
import pandas as pd
import numpy as np
import warnings
from scipy.stats import randint, uniform, zscore

'''Visualization Imports'''
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from prettytable import PrettyTable

'''Import Data'''
from sklearn.datasets import load_digits, fetch_california_housing

'''Import Data Processing Utilities'''
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

'''Import Predictors'''
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

'''Import Model Tuning Utilities'''
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

'''Import Metrics'''
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    mean_squared_error, 
    mean_absolute_error, 
    r2_score, 
    explained_variance_score, 
    max_error, 
    mean_squared_log_error, 
    median_absolute_error, 
    mean_absolute_percentage_error, 
    mean_poisson_deviance,
    mean_gamma_deviance
)

'''Suppress Warnings'''
warnings.filterwarnings("ignore")

In [ ]:
'''Define Global Variables'''

'''Define Helper Functions'''



In [ ]:
'''Import Classifier data'''
digits = load_digits()


In [ ]:
'''Data Visualization and EDA'''

print('--- DIGITS EDA ---')

# Load digits dataset
digits = load_digits()

# Convert to DataFrame for easier manipulation
df_digits = pd.DataFrame(digits.data, columns=[f'Pixel_{i}' for i in range(digits.data.shape[1])])
df_digits['Target'] = digits.target

print(f'\nMissing Values: {df_digits.isna().sum().sum()}'
      f'\nDuplicates: {df_digits.duplicated().sum()}'
      f'\nShape: {df_digits.shape}')

print(df_digits.info())

df_digits.describe()

'''# Distribution of Target Digits
target_counts = df_digits['Target'].value_counts()
plt.figure(figsize=(8, 5))
sns.barplot(x=target_counts.index, y=target_counts.values, palette='viridis')
plt.title("Distribution of Target Digits")
plt.xlabel("Digits")
plt.ylabel("Frequency")
plt.show()'''

# Correlation Heatmap (First 20 Pixels)
correlation_matrix = df_digits.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix.iloc[:20, :20], cmap='coolwarm', cbar=True, vmin=-1, vmax=1)
plt.title("Correlation Heatmap (First 20 Pixels)")
plt.show()
'''
# Feature Variance and Low-Variance Features
feature_variances = df_digits.var().drop('Target')
plt.figure(figsize=(10, 5))
sns.lineplot(x=range(len(feature_variances)), y=feature_variances, color='blue')
plt.title("Feature Variances")
plt.xlabel("Feature Index")
plt.ylabel("Variance")
plt.show()

low_variance_features = feature_variances[feature_variances < 0.1]
print("\nLow-variance features:\n", low_variance_features)
'''
# PCA and 2D Visualization
pca = PCA(n_components=2)
digits_pca = pca.fit_transform(digits.data)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=digits_pca[:, 0], y=digits_pca[:, 1], hue=digits.target, palette='tab10', s=60)
plt.title("2D PCA Projection of Digits Dataset")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Digits", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
'''
# Distribution of Pixel Values for Digit '0'
digit_zero = df_digits[df_digits['Target'] == 0]
plt.figure(figsize=(12, 6))
for i in range(5):
    sns.kdeplot(digit_zero[f'Pixel_{i}'], label=f'Pixel_{i}', fill=True)
plt.title("Distribution of Pixel Values for Digit '0'")
plt.xlabel("Pixel Value")
plt.ylabel("Density")
plt.legend()
plt.show()

# Pairplot of Selected Pixels
selected_pixels = ['Pixel_0', 'Pixel_10', 'Pixel_20', 'Pixel_30', 'Target']
sns.pairplot(df_digits[selected_pixels], hue='Target', palette='tab10')
plt.suptitle("Pairplot of Selected Pixels", y=1.02, fontsize=14)
plt.show()
'''
# Average Pixel Intensity for Each Digit
mean_pixel_intensity = df_digits.groupby('Target').mean().iloc[:, :64]
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.ravel()):
    digit_heatmap = mean_pixel_intensity.iloc[i].values.reshape(8, 8)
    sns.heatmap(digit_heatmap, ax=ax, cmap='viridis', cbar=False, annot=False)
    ax.set_title(f"Digit: {i}")
    ax.axis('off')
plt.suptitle("Average Pixel Intensity for Each Digit", fontsize=16)
plt.show()

print('--- CALIFORNIA HOUSING EDA ---')

# Load California Housing dataset
housing = fetch_california_housing(as_frame=True)
X_housing = housing.data
y_housing = housing.target
df_housing = pd.concat([X_housing, y_housing.rename('MedHouseVal')], axis=1)

# Display dataset information
'''print("\nColumn Descriptions:\n", housing.feature_names + ["MedHouseVal (Target)"])
df_housing.info()
if df_housing.isnull().sum().sum() == 0:
    print("\nNo missing values in the dataset!")'''

print(f'\nMissing Values: {df_housing.isna().sum().sum()}'
      f'\nDuplicates: {df_housing.duplicated().sum()}'
      f'\nShape: {df_housing.shape}')

# Apply log transformation to median house value
df_housing['MedHouseVal_log'] = np.log1p(df_housing['MedHouseVal'])

# Create a new quintile column for Median Income
df_housing['MedInc_quintile'] = pd.qcut(df_housing['MedInc'], 5)

# Set a more academic style for plots
sns.set_theme(style="whitegrid", palette="muted", font_scale=1.2)

# Create subplots with a single call
fig, axes = plt.subplots(3, 2, figsize=(15, 15))
plt.subplots_adjust(hspace=0.7, wspace=0.4)

# Flatten axes for easy iteration
axes = axes.ravel()

# Pre-calculate column names and avoid repeated access to df.columns
columns = ['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'MedHouseVal']

# Loop through columns and plot
for i, col in enumerate(columns):
    try:
        # Drop NaN values and plot the histogram with KDE
        sns.histplot(df_housing[col].dropna(), kde=True, bins=25, ax=axes[i])
        axes[i].set_title(f'Frequency Distribution of {col}')
        axes[i].set_xlabel('Values')
        axes[i].set_ylabel('Frequency')

    except Exception as e:
        # Skip problematic columns without excessive printouts
        print(f"Could not plot {col}: {e}")
        continue

# Add the Log-Transformed Median House Value by Median Income Quintiles plot
try:
    sns.boxplot(
        x='MedInc_quintile',
        y='MedHouseVal_log',
        data=df_housing,
        palette="muted",
        ax=axes[len(columns)]
    )
    axes[len(columns)].set_title("Log-Transformed Median House Value by Median Income Quintiles")
    axes[len(columns)].set_xlabel("Median Income (Quintiles)")
    axes[len(columns)].set_ylabel("Log Median House Value ($100,000)")
    axes[len(columns)].tick_params(axis='x', rotation=45)
except Exception as e:
    print(f"Could not plot Log-Transformed Median House Value: {e}")

# Hide remaining unused axes
for j in range(len(columns) + 1, len(axes)):
    axes[j].axis('off')  # This method is generally faster than set_visible(False)

df_housing = df_housing.drop(['MedHouseVal_log','MedInc_quintile'], axis=1)

# Final adjustments and display
plt.tight_layout()
plt.show()

# Correlation Heatmap
correlation_matrix = df_housing.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, vmin=-1, vmax=1)
plt.title("Correlation Heatmap with Adjusted Scale")
plt.show()

In [ ]:
'''Instantiate and Evaluate Default Classifiers'''
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3, random_state=42)
for model in [DecisionTreeClassifier, RandomForestClassifier]:
    pipe = Pipeline([
        ('scale', StandardScaler()),
        # ('pca', PCA()), PCA Unused due to negative impact on model performance
        ('classify',model())],
    verbose=True)
    predicted = pipe.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_test, predicted))
    sns.heatmap(confusion_matrix(y_test, predicted))
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
'''Instantiate and Tune Classifiers'''

classifiers = {
    DecisionTreeClassifier.__name__: {
        'model': DecisionTreeClassifier,
        'paramSpace': {
            'classify__criterion': Categorical(['gini', 'entropy', 'log_loss']),
            'classify__splitter': Categorical(['best', 'random']),
            'classify__max_depth': Integer(1, 1000),
            'classify__min_samples_split': Real(0.01, 0.9),
            'classify__min_samples_leaf': Real(0.01, 0.9),
            'classify__min_weight_fraction_leaf': Real(0.0,0.5),
            'classify__max_features': Real(0.01,0.9),
            'classify__max_leaf_nodes': Integer(2, 4000), 
            'classify__min_impurity_decrease': Real(0.0, 1.0),
            'classify__ccp_alpha': Real(0.01, 0.9),
            'pca__n_components': Integer(1,len(digits.data[0])),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False],
        }
    },
    
    RandomForestClassifier.__name__: {
        'model': RandomForestClassifier,
        'paramSpace': {
            'classify__n_estimators': Integer(10, 2000),
            'classify__criterion': Categorical(['gini', 'entropy', 'log_loss']),
            'classify__max_depth': Integer(1, 1000), 
            'classify__min_samples_split': Real(0.01, 0.9), 
            'classify__min_samples_leaf': Real(0.01, 0.9),
            'classify__min_weight_fraction_leaf': Real(0.01,0.5),
            'classify__max_features': Real(0.01,0.9),
            'classify__max_leaf_nodes': Integer(1,2000),
            'classify__min_impurity_decrease': Real(0.01,0.9),
            # 'classify__bootstrap': Categorical([True, False]),
            'classify__oob_score': Categorical([True, False]),
            'classify__warm_start': Categorical([True, False]),
            'classify__max_samples':Real(0.01,0.9),
            'pca__n_components': Integer(1,len(digits.data[0])),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False],
        }
    }
}

tunedModels = {}

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3, random_state=42)
for name, classDict in classifiers.items():
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA()),
        ('classify',classDict['model']())],
    verbose=False)

    tunedModels[name] = BayesSearchCV(
        pipe,
        classDict['paramSpace'],
        n_iter= 5,#30, # Reduced for performance during development
        cv= 4,#20, # Reduced for performance during development
        scoring='accuracy',
        # TODO: Use GridSearch for scoring criteria
        # NOTE: This will take 13 hours. Execute over night.
        random_state=42
        )
    tunedModels[name].fit(X_train, y_train)
    
    # search = BayesSearchCV(pipe, param_grid, n_jobs=2)
    # predicted = search.best_estimator_.predict(X_test)
    predicted = tunedModels[name].best_estimator_.predict(X_test)
    
    print(classification_report(y_test, predicted))
    sns.heatmap(confusion_matrix(y_test, predicted))
    plt.title(name + ' Confusion Matrix')
    plt.show()




In [ ]:
'''Evaluate tuning process and resultant models'''


In [ ]:
'''Import Regression Data'''
cal_housing = fetch_california_housing(as_frame=True)



In [ ]:
'''Initial data EDA'''

outlierTable = PrettyTable(['Feature', 'Outlier Count'])

for column in cal_housing.data.columns:
    Q1 = cal_housing.data[column].quantile(0.25)
    Q3 = cal_housing.data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    outlierCount = np.array(cal_housing.data[column] >= upper).sum() + np.array(cal_housing.data[column] <= lower).sum()
    outlierTable.add_row([column, outlierCount])
    # print(f"{column}: {outlierCount}")

print(outlierTable)


In [ ]:
'''Data Cleaning and Pre-Processing'''
cleanData = cal_housing.data
cleanData['y'] = cal_housing.target

cleanData = cleanData.drop(columns=['Longitude', 'Latitude'])
for feature in ['AveBedrms', 'AveRooms', 'AveOccup', 'Population']:
    cleanData = cleanData[(np.abs(zscore(cleanData[feature])) < 2.5)]
cleanTarget = cleanData['y'].to_list()

# DATA CLEANING TODOs
# TODO: Bin Lat/Long groupings into city/town clusters. look for available geo-fencing data for cluster labeling - can we do a graph of centroids on top of map?
# TODO: Fix Skew for Population, MedIncome, AvgOccup, AvgBedroom, Target
# TODO: Feature Engineering / Reduction
cleanData.drop(columns=['y'], inplace=True)

In [ ]:
'''Instantiate and Evaluate Default Regressors'''
modelData = cleanData.copy()

# TODO: How does normalization vs standardization impact model performance
transformPipeline = [
    ('scaler', StandardScaler()),
    # ('feature_reduction', PCA(n_components=5,iterated_power=7))
    ]

X_train, X_test, y_train, y_test = train_test_split(modelData, cleanTarget, test_size=0.3, random_state=42)
for regressor in [DecisionTreeRegressor, RandomForestRegressor]:
    pipe = Pipeline(transformPipeline + [('regress',regressor())], verbose=True)
    predicted = pipe.fit(X_train, y_train).predict(X_test)
    mse = mean_squared_error(y_test, predicted)
    print(regressor.__name__)
    print(mse)
    
    
    # TODO: Graph Regression Plane using skopt.plots
    # NOTE: try using PCA to force data into 3d space
    

In [ ]:
'''Instantiate and Evaluate Default Regressors'''
modelData = cleanData.copy()

# TODO: How does normalization vs standardization impact model performance
transformPipeline = [
    ('scaler', MinMaxScaler()),
    # ('feature_reduction', PCA(n_components=5,iterated_power=7))
    ]

X_train, X_test, y_train, y_test = train_test_split(modelData, cleanTarget, test_size=0.3, random_state=42)
for regressor in [DecisionTreeRegressor, RandomForestRegressor]:
    pipe = Pipeline(transformPipeline + [('regress',regressor())], verbose=True)
    predicted = pipe.fit(X_train, y_train).predict(X_test)
    mse = mean_squared_error(y_test, predicted)
    print(regressor.__name__)
    print(mse)
    
    
    # TODO: Graph Regression Plane using skopt.plots
    # NOTE: try using PCA to force data into 3d space
    

In [ ]:
'''Instantiate and Tune Regressors'''
# TODO: Explore how increased demetionality in the parameter space impacts optimization performance
regressors = {
    DecisionTreeRegressor.__name__: {
        'model': DecisionTreeRegressor,
        'paramSpace': {
            'regress__criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
            'regress__splitter': ['best', 'random'],
            'regress__max_depth': Integer(2, 1000),
            'regress__min_samples_split': Real(0.01, 0.9),
            'regress__min_samples_leaf': Real(0.01, 0.9),
            'regress__min_weight_fraction_leaf': Real(0.0, 0.5),
            'regress__max_features': Real(0.01, 0.5),
            'regress__max_leaf_nodes': Integer(2, 1000),
            'regress__min_impurity_decrease': Real(0.0, 0.9),
            'regress__ccp_alpha': Real(0.01, 0.9),
            'pca__n_components': Integer(1,len(modelData.columns)),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False],
        }
    },
    RandomForestRegressor.__name__: {
        'model': RandomForestRegressor,
        'paramSpace': {
            'regress__n_estimators': Integer(50, 500),
            'regress__criterion': Categorical(['squared_error', 'friedman_mse', 'absolute_error', 'poisson']),
            'regress__max_depth': Integer(2, 1000), 
            'regress__min_samples_split': Real(0.01, 0.9),
            'regress__min_samples_leaf': Real(0.01, 0.9),
            'regress__min_weight_fraction_leaf': Real(0.01, 0.5),
            'regress__max_features': Real(0.01,0.9),
            # 'regress__max_features': Categorical(['sqrt', 'log2']), 
            'regress__max_leaf_nodes': Integer(2,1000),
            'regress__min_impurity_decrease': Real(0.01, 0.9),
            # 'regress__bootstrap': [True, False],
            'regress__oob_score': [True, False],
            'regress__warm_start': [True, False],
            'regress__ccp_alpha': Real(0.01, 0.9),
            'regress__max_samples': Real(0.01, 0.9),
            'pca__n_components': Integer(1,len(modelData.columns)),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False],
        }
    }
}

tunedModel = {}
X_train, X_test, y_train, y_test = train_test_split(modelData, cleanTarget, test_size=0.3, random_state=42)

for name, regDict in regressors.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA()),
        ('regress',regDict['model']())])
    tunedModel[name] = BayesSearchCV(
        pipe,
        regDict['paramSpace'],
        n_iter=5,#30,
        cv=5,#20,
        # scoring=scoreModel
        # scoring = scoringCriteria[i]
        # TODO: Use GridSearchCV for scoringCriteria param space
        # NOTE: This will take 13 hours. Execute over night.
        )
    tunedModel[name].fit(X_train, y_train)
    predicted = tunedModel[name].best_estimator_.predict(X_test)
    print('Evaluation Metric:', tunedModel[name].get_params()['scoring'])
    print("val. score: %s" % tunedModel[name].best_score_)
    print("test score: %s" % tunedModel[name].score(X_test, y_test))
    

## NEW CODE BELOW!! - Will

In [ ]:
scalers = {'Standardization': StandardScaler(), 'Normalization': MinMaxScaler()}
X_train, X_test, y_train, y_test = train_test_split(modelData, cleanTarget, test_size=0.3, random_state=42)

for scale_name, scaler in scalers.items():
    print(f"Testing with {scale_name}")
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    for regressor in [DecisionTreeRegressor, RandomForestRegressor]:
        # Define and fit pipeline
        pipe = Pipeline([('scaler', scaler), ('regress', regressor())], verbose=True)
        predicted = pipe.fit(X_train, y_train).predict(X_test)
        y_test_np = np.array(y_test)
        
        # Plot Actual vs Predicted
        plt.figure(figsize=(10, 5))
        plt.scatter(y_test, predicted, alpha=0.7, color="blue", label="Predicted vs Actual")
        plt.plot([y_test_np.min(), y_test_np.max()], [y_test_np.min(), y_test_np.max()], color="red", lw=2, label="Ideal")
        plt.title(f"Actual vs Predicted - {regressor.__name__}")
        plt.xlabel("Actual")
        plt.ylabel("Predicted")
        plt.legend()
        plt.show()
        
        # Plot Residuals
        residuals = y_test - predicted
        plt.figure(figsize=(10, 5))
        plt.scatter(predicted, residuals, alpha=0.7, color="purple")
        plt.axhline(0, color="red", linestyle="--")
        plt.title(f"Residuals - {regressor.__name__}")
        plt.xlabel("Predicted")
        plt.ylabel("Residuals")
        plt.show()

        # Apply PCA for 3D Plot
        pca = PCA(n_components=2)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)
        
        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(X_test_pca[:, 0], X_test_pca[:, 1], y_test_np, color='blue', label='Actual')
        ax.scatter(X_test_pca[:, 0], X_test_pca[:, 1], predicted, color='green', alpha=0.7, label='Predicted')
        ax.set_title("3D PCA Regression Plot")
        ax.set_xlabel("Principal Component 1")
        ax.set_ylabel("Principal Component 2")
        ax.set_zlabel("Output")
        ax.legend()
        plt.show()

        # Calculate and print metrics
        print('Mean Absolute Error (MAE):', mean_absolute_error(y_test, predicted))
        print('Mean Squared Error (MSE):', mean_squared_error(y_test, predicted))                           # preferred metric
        print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(y_test, predicted)))
        print('Mean Absolute Percentage Error (MAPE):', mean_absolute_percentage_error(y_test, predicted))
        print('Explained Variance Score:', explained_variance_score(y_test, predicted))
        print('Max Error:', max_error(y_test, predicted))
        print('Mean Squared Log Error:', mean_squared_log_error(y_test, predicted))
        print('Median Absolute Error:', median_absolute_error(y_test, predicted))
        print('R^2:', r2_score(y_test, predicted))                                                          # preferred metric
        print('Mean Poisson Deviance:', mean_poisson_deviance(y_test, predicted))
        print('Mean Gamma Deviance:', mean_gamma_deviance(y_test, predicted))
        print('-' * 50)  # Separator between different model results


In [ ]:
# Classification Metrics
classification_metrics = [
    'accuracy', 'f1_macro', 'roc_auc', 'balanced_accuracy',
    'precision_macro', 'recall_macro', 'average_precision'
]

# Filter only regression-compatible scoring metrics
regression_scoring_criteria = [
    'neg_mean_squared_error', 'r2', 'neg_mean_absolute_error',
    'neg_mean_squared_log_error', 'neg_median_absolute_error',
    'max_error', 'explained_variance', 'neg_root_mean_squared_error'
]


# Define scoring criteria
scoring_criteria = {
    'classification': ['accuracy', 'f1_macro', 'roc_auc', 'balanced_accuracy', 'precision_macro', 'recall_macro', 'average_precision'],
    'regression': [
        'neg_mean_squared_error', 'r2', 'neg_mean_absolute_error', 'neg_mean_squared_log_error', 
        'neg_median_absolute_error', 'max_error', 'explained_variance', 'neg_root_mean_squared_error'
    ]
}

# Regressors parameter spaces for GridSearchCV and RandomizedSearchCV
param_spaces = {
    'DecisionTreeRegressor': {
        'randomized': {
            'regress__criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
            'regress__splitter': ['best', 'random'],
            'regress__max_depth': randint(2, 101),
            'regress__min_samples_split': uniform(0.01, 0.89),
            'regress__min_samples_leaf': uniform(0.01, 0.89),
            'regress__max_features': uniform(0.01, 0.49),
            'regress__ccp_alpha': uniform(0.01, 0.89),
            'pca__n_components': randint(1, 100),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False]
        },
        'grid': {
            'regress__splitter': ['best', 'random'],
            'regress__max_depth': list(range(2, 1001)),
            'regress__min_samples_split': [i / 100 for i in range(1, 91)],
            'regress__min_samples_leaf': [i / 100 for i in range(1, 91)],
            'regress__max_features': [i / 100 for i in range(1, 51)],
            'regress__ccp_alpha': [i / 100 for i in range(1, 91)],
            'pca__n_components': list(range(1, 101)),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False]
        }
    },
    'RandomForestRegressor': {
        'randomized': {
            'regress__n_estimators': randint(50, 501),
            'regress__max_depth': randint(2, 101),
            'regress__min_samples_split': uniform(0.01, 0.89),
            'regress__min_samples_leaf': uniform(0.01, 0.89),
            'regress__max_features': uniform(0.01, 0.89),
            'regress__ccp_alpha': uniform(0.01, 0.89),
            'regress__max_samples': uniform(0.01, 0.89),
            'pca__n_components': randint(1, 100),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False]
        },
        'grid': {
            'regress__n_estimators': list(range(50, 501)),
            'regress__max_depth': list(range(2, 1001)),
            'regress__min_samples_split': [i / 100 for i in range(1, 91)],
            'regress__min_samples_leaf': [i / 100 for i in range(1, 91)],
            'regress__max_features': [i / 100 for i in range(1, 91)],
            'regress__ccp_alpha': [i / 100 for i in range(1, 91)],
            'regress__max_samples': [i / 100 for i in range(1, 91)],
            'pca__n_components': list(range(1, 101)),
            'scaler__with_mean': [True, False],
            'scaler__with_std': [True, False]
        }
    }
}

# Function to perform GridSearchCV or RandomizedSearchCV
def tune_model(search_type, model_name, model, param_space, X_train, y_train, X_test, y_test):
    pipe = Pipeline([ 
        ('scaler', StandardScaler()), 
        ('pca', PCA()), 
        ('regress', model()) 
    ])

    # Set search class based on the search_type
    search_class = RandomizedSearchCV if search_type == 'randomized' else GridSearchCV

    # Set parameters based on search type
    search_params = {
        'estimator': pipe,
        'n_jobs': -1,
        'cv': 5,
        'scoring': None,
        'return_train_score': True
    }

    # For RandomizedSearchCV, use 'param_distributions' and set 'n_iter'
    if search_type == 'randomized':
        search_params['n_iter'] = 50  # Only for RandomizedSearchCV
        search_params['param_distributions'] = param_space  # Use for RandomizedSearchCV
    else:
        search_params['param_grid'] = param_space  # Use param_grid for GridSearchCV

    search = search_class(**search_params)

    # Loop over scoring metrics for evaluation
    for scoring_metric in scoring_criteria['regression']:
        print(f'Tuning {model_name} with {search_type} search and scoring metric: {scoring_metric}')
        search.set_params(scoring=scoring_metric)
        search.fit(X_train, y_train)
        
        print(f'Evaluation Metric: {scoring_metric}')
        print(f"Best Score (Train): {search.best_score_}")
        print(f"Test Score: {search.score(X_test, y_test)}")
        print('-' * 80)

    return search

# Split the data
X_train, X_test, y_train, y_test = train_test_split(modelData, cleanTarget, test_size=0.3, random_state=42)

# Instantiate tuned models dictionary
tuned_models = {}

# Iterate through each regressor and perform tuning
for model_name, model_dict in param_spaces.items():
    # Run RandomizedSearchCV first
    if 'randomized' in model_dict:
        param_space = model_dict['randomized']
        search = tune_model('randomized', model_name, globals()[model_name], param_space, X_train, y_train, X_test, y_test)
        tuned_models[model_name] = search

    # Then run GridSearchCV
    if 'grid' in model_dict:
        param_space = model_dict['grid']
        search = tune_model('grid', model_name, globals()[model_name], param_space, X_train, y_train, X_test, y_test)
        tuned_models[model_name] = search


In [ ]:
regressor = opt.best_estimator_.steps[-1][1]
features = modelData.columns
importance = regressor.feature_importances_

plt.figure(figsize=(10, 6))
plt.barh(list(modelData.columns), importance)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances')
plt.show()

In [ ]:
'''Evaluate tuning process and resultant models'''

